In [ ]:
import pyzx as zx
from fractions import Fraction

In [ ]:
# Uses the following identity:
#         ───W(a)───W(b)───
#         ≡ ───Z^-a───X───Z^a───Z^-b───X───Z^b───
#         ≡ ───Z^-a───Z^-a───Z^b───X───X───Z^b───
#         ≡ ───Z^-a───Z^-a───Z^b───Z^b───
#         ≡ ───Z^2(b-a)───

# ──W(a)─ ≡ ───Z^-a───X───Z^a─
def W(a):
    c = zx.Circuit(qubit_amount=1)
    c.add_gate("ZPhase", 0, "-" + a)
    c.add_gate("NOT", 0)
    c.add_gate("ZPhase", 0, a)
    return c

In [ ]:
# zx.draw(W("1/7"))

c = W("1/7") + W("1/3")
# zx.draw(c)
g = c.to_graph()
# zx.full_reduce(g, quiet=False)
zx.draw(g)
print(g.types())

In [ ]:
c = zx.Circuit(qubit_amount=3)
c.add_gate("HAD", 1)
c.add_gate("CNOT", 1, 2)
c.add_gate("CNOT", 1, 0)
c.add_gate("HAD", 1)
c.add_gate("CZ", 1, 2)
c.add_gate("CX", 0, 2)
print (c.col_add())
zx.draw(c)

g = c.to_graph()
zx.full_reduce(g)
zx.draw(g)

In [ ]:
def mkgraph():
    gr = zx.Graph()
    l = gr.add_vertex(zx.VertexType.BOUNDARY)
    a = gr.add_vertex()
    b = gr.add_vertex()
    r = gr.add_vertex(zx.VertexType.BOUNDARY)
    gr.add_edges([(l, a), (a, b), (b, r)])
    
    zx.draw(gr)

mkgraph()

In [ ]:
def parseMLIR(filename):
    values = dict()
    g = zx.Graph('simple')
    n = 0
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.strip().split(' ')
            if len(line) == 0:
                continue
            if line[0] == 'zxg.wire':
                g.add_edge((values[line[1]], values[line[2]]))
                continue
            if line[0][0:1] == '%':
                if line[2] == 'constant':
                    values[line[0]] = float(line[3])
                if line[2] == 'zxg.H':
                    values[line[0]] = g.add_vertex(zx.VertexType.H_BOX, n, n)
                if line[2] == 'zxg.Z':
                    values[line[0]] = g.add_vertex(zx.VertexType.Z, n,n, phase=Fraction(2,1))
                if line[2] == 'zxg.X':
                    values[line[0]] = g.add_vertex(zx.VertexType.X, n,n,phase=Fraction(2,1))
                    g.set_phase(values[line[0]], 1)
                if line[2] == 'zxg.terminal':
                    values[line[0]] = g.add_vertex(zx.VertexType.BOUNDARY)
                    g.inputs.append(values[line[0]])
                n += 1
    return g

In [ ]:
gr = parseMLIR('../test/ZXGraph/Rewrites/entanglement-detection.mlir')
# zx.full_reduce(gr)
zx.draw(gr)
print(gr.vertices())
print(list(gr.edges()))
print(gr.phases())